In [2]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
paths = {
    "clockwise": r"D:\Graduation Project\data\hand_gesture_project\data\colck.csv",
    "anticlockwise": r"D:\Graduation Project\data\hand_gesture_project\data\inverse.csv",
    "updown": r"D:\Graduation Project\data\hand_gesture_project\data\up_down.csv"
}

dfs = []
for label, path in enumerate(paths.values()):
    df = pd.read_csv(path, sep=";")
    df["label"] = label
    dfs.append(df)
df_all = pd.concat(dfs, ignore_index=True)

In [4]:
# cleaning data
for col in ["Accel X", "Accel Y", "Accel Z", "Gyro X", "Gyro Y", "Gyro Z"]:
    df_all[col] = pd.to_numeric(df_all[col], errors='coerce')
df_all = df_all.dropna().reset_index(drop=True)

In [5]:
# Energy
df_all["energy"] = df_all[["Accel X", "Accel Y", "Accel Z", "Gyro X", "Gyro Y", "Gyro Z"]].pow(2).sum(axis=1)
df_all["energy_smooth"] = df_all["energy"].rolling(window=10, center=True).mean()


In [6]:
# knowing peaks
peaks, _ = find_peaks(df_all["energy_smooth"], distance=50, prominence=5)


In [7]:
window_size = 50
sequences, labels = [], []

for peak in peaks:
    start = max(peak - window_size // 2, 0)
    end = min(peak + window_size // 2, len(df_all))
    if end - start == window_size:
        segment = df_all.iloc[start:end][["Accel X", "Accel Y", "Accel Z", "Gyro X", "Gyro Y", "Gyro Z"]].values
        sequences.append(segment)
        labels.append(df_all.iloc[start:end]["label"].mode()[0])  # أكتر label متكرر

X = np.array(sequences)
y = np.array(labels)

In [8]:
scaler = StandardScaler()
X_scaled = X.reshape(-1, 6)
X_scaled = scaler.fit_transform(X_scaled).reshape(X.shape)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)


In [9]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(window_size, 6)))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y_encoded)), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 128)           69120     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 120707 (471.51 KB)
Trainable params: 120707 (471.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[es])


Epoch 1/30


10/10 [==============================] - 5s 149ms/step - loss: 0.8307 - accuracy: 0.8323 - val_loss: 0.3909 - val_accuracy: 1.0000
Epoch 2/30
10/10 [==============================] - 0s 42ms/step - loss: 0.2156 - accuracy: 0.9937 - val_loss: 0.1269 - val_accuracy: 0.9873
Epoch 3/30
10/10 [==============================] - 0s 40ms/step - loss: 0.0406 - accuracy: 1.0000 - val_loss: 0.1566 - val_accuracy: 0.9747
Epoch 4/30
10/10 [==============================] - 0s 40ms/step - loss: 0.0258 - accuracy: 0.9968 - val_loss: 0.1739 - val_accuracy: 0.9747
Epoch 5/30
10/10 [==============================] - 0s 41ms/step - loss: 0.0253 - accuracy: 0.9968 - val_loss: 0.2712 - val_accuracy: 0.9620
Epoch 6/30
10/10 [==============================] - 0s 40ms/step - loss: 0.0249 - accuracy: 0.9968 - val_loss: 0.2862 - val_accuracy: 0.9620
Epoch 7/30
10/10 [==============================] - 0s 38ms/step - loss: 0.0239 - accuracy: 0.9968 - val_loss: 0.2973 - val_accuracy: 0.9620


In [11]:
# الفكرة: لو الثقة (probability) أقل من عتبة معينة، متعملش prediction
def safe_predict(model, sample, threshold=0.7):
    probs = model.predict(sample.reshape(1, window_size, 6))[0]
    if np.max(probs) < threshold:
        return "Unknown Movement ❌"
    else:
        return le.inverse_transform([np.argmax(probs)])[0]


In [12]:
import joblib
joblib.dump(scaler, r"D:\Graduation Project\data\hand_gesture_project\models\scaler2.pkl")
joblib.dump(le, r"D:\Graduation Project\data\hand_gesture_project\models\label_encoder2.pkl")

from tensorflow.keras.models import load_model

# حفظ الموديل بصيغة HDF5 (المفضلة مع Keras)
model.save(r"D:\Graduation Project\data\hand_gesture_project\models\gesture_lstm_model.h5")

c:\Users\hp zbook\.conda\envs\gess_env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
